In [5]:
### Academic Conntect
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import TFAutoModel, AutoTokenizer


In [11]:
# Read university dataset 
uni_data = pd.read_excel("university_data.xlsx")
uni_data.head()

,Student GUID,Name,Research Interests,University Field
0,c1c5fc27-048a-4fad-b32a-57b6613f5c6d,Daniel Cain,"Photonics, Cosmology, Theoretical Physics, Exp...",Physics
1,ead3d7a5-bddc-4ad1-ab55-4db006731802,Amy Potter,"Cognitive Psychology, Developmental Psychology...",Psychology
2,c6f1e6d1-21fe-4daa-a022-ff9e0f4fd957,Jessica Collins,"Materials Science, Physical Chemistry, Inorgan...",Chemistry
3,3e19f76f-46b4-46c4-a489-36053fd8d79e,Maria Singh,"Economic History, History of Science, Military...",History
4,31bbb063-8dae-4e81-97c4-456e8df9af33,James Thomas,"Geometry, Mathematical Physics, Statistics, Al...",Mathematics


In [32]:
from sklearn.preprocessing import LabelEncoder

# Cleaning up Research Interests feature
uni_data['Research Interests'] = uni_data['Research Interests'].str.lower().str.replace(r'[^\w\s]+', '')

# Cleaning up "University Field" feature
# uni_data["University Field"] = uni_data["University Field"].str.lower().str.replace(r'[^\w\s]+', '')
# uni_data.head()
encoder = LabelEncoder()
uni_data['University Field'] = encoder.fit_transform(uni_data['University Field'])

In [28]:
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModel.from_pretrained('bert-base-uncased')

def embed_text(texts):
    inputs = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=512, return_tensors="tf")
    outputs = model(inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

# Embed 'Research Interests'
embeddings = embed_text(uni_data['Research Interests'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [30]:
embeddings

array([[ 0.35122058,  0.2672667 , -0.17760639, ..., -0.17172086,
        -0.18274336,  0.6307969 ],
       [ 0.13852608,  0.29572952, -0.7052915 , ..., -0.30475768,
        -0.17106837,  0.39814553],
       [ 0.34270325,  0.21112193, -0.3461482 , ..., -0.23749661,
        -0.01269638,  0.6580549 ],
       ...,
       [-0.03102134, -0.03761458, -0.58762026, ..., -0.3957855 ,
        -0.20575732,  0.42250288],
       [ 0.49557954,  0.16372317, -0.3434162 , ..., -0.19094762,
         0.10214731,  0.82497257],
       [ 0.49353784,  0.18878661, -0.33193555, ..., -0.21652126,
        -0.00542787,  0.8533199 ]], dtype=float32)

In [33]:
from sklearn.preprocessing import StandardScaler

# Scale embeddings and the encoded field
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)
field_scaled = scaler.fit_transform(uni_data[['University Field']])

# Combine embeddings with the university field
combined_features = np.hstack((embeddings_scaled, field_scaled))

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(combined_features)

# Optionally, create a DataFrame to view similarity scores
similarity_df = pd.DataFrame(similarity_matrix, index=uni_data['Student GUID'], columns=uni_data['Student GUID'])

In [37]:
def get_top_matches(student_name, similarity_df, top_n=5):
    scores = similarity_df.loc[student_name]
    top_matches = scores.sort_values(ascending=False)[1:top_n+1]  # exclude self-match
    return top_matches

# Example usage
top_matches = get_top_matches('c1c5fc27-048a-4fad-b32a-57b6613f5c6d', similarity_df)
print(top_matches)

Student GUID
149b3317-e913-4008-bd23-7a2f445b6f0b    0.988040
ac2087b4-ef74-4b5e-abad-5302f9d7e368    0.987624
adb2fcac-7d96-47ec-8f89-29a6ec92b599    0.984967
a70f1a6e-2bc8-4060-b3a4-f0c3045e769c    0.984064
7aeaf1f2-d2a4-4176-abdf-96212da216d4    0.982235
Name: c1c5fc27-048a-4fad-b32a-57b6613f5c6d, dtype: float64


In [18]:
# max_len = 0
# for i in uni_data["Research Interests"]:
#     if len(i.split()) > max_len:
#         max_len = len(i.split())

# print(max_len)


21


In [19]:
# from tensorflow.keras.layers import TextVectorization 

# # Use the default TextVectorization variables
# text_vectorizer = TextVectorization(max_tokens=max_len, # how many words in the vocabulary (all of the different words in your text)
#                                     standardize="lower_and_strip_punctuation", # how to process text
#                                     split="whitespace", # how to split tokens
#                                     ngrams=None, # create groups of n-words?
#                                     output_mode="int", # how to map tokens to numbers
#                                     output_sequence_length=None,
#                                     pad_to_max_tokens=True)

In [20]:
# max_vocab_length = 10000

# text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
#                                     output_mode="int",
#                                     output_sequence_length=max_len)